In [45]:
import pandas as pd
import numpy as np
import altair as alt
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error,make_scorer

#Comment out for other models
# from sklearn.linear_model import Lasso 
# from sklearn.linear_model import Ridge 
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import KFold


In [20]:
pd.set_option('max_columns', None)
# ##### Sample teams for streamlit
# sample_teams = ["Michigan","Duke","North Carolina","Kansas","Gonzaga","Purdue","Kentucky","Auburn","Tennessee","Baylor",'Texas Tech',"Villanova","Wisconsin"]

##### Uncomment for streamlit sample 
# sample_df_box_score_and_features= pd.read_csv("src/sample_player_and_opp_stats_with_position.csv",index_col = 0)
sample_df_box_score_and_features=pd.read_csv("src\\Features_and_boxscores\\limited_sampled_player_and_opp_stats_with_position.csv",index_col= 0)


In [31]:
pd.options.mode.chained_assignment = None
### Using 2022 as test year. Train on a sample of the other years  
training_df =  sample_df_box_score_and_features[sample_df_box_score_and_features["season"]<2022].sample(5000,random_state=698)
testing_df = sample_df_box_score_and_features[sample_df_box_score_and_features["season"]==2022]
### fill in colors for missing teams - Dark Grey 
testing_df["team_color"] = testing_df.loc[:,"team_color"].fillna(value='#555555')



In [32]:
points_model_df = testing_df[testing_df["team_short_display_name"]== "Michigan"]
points_model_df

,team_short_display_name,game_id,starting_lineup,athlete_display_name,min,oreb,dreb,reb,ast,stl,blk,to,pf,pts,starter,ejected,did_not_play,active,athlete_jersey,athlete_id,athlete_short_name,athlete_headshot_href,athlete_position_name,athlete_position_abbreviation,team_name,team_logo,team_id,team_abbreviation,team_color,season,season_type,game_date,FGM,FGA,3PM,3PA,FTM,FTA,largest_lead,home_away,opponent_id,opponent_abbrev,points_per_min,prev_player_ppg,prev_player_points_per_min,game_total_points,game_number,Cleaned_position_bigs_guards,cum_average_opp_points_to_role,cum_average_opp_min_allowed_to_role,prev_3_games_opp_min_allowed_to_role,prev_3_games__opp_points_allowed_to_role,total_pts_allowed,prev_total_opp_ppg,Home,Starter_numeric,prev_avg_pts_to_role_pos,prev_avg_min_to_role_position,ATH,Big_man,Guard
19215,Michigan,401364421,"{""DeVante' Jones"", 'Moussa Diabate', 'Eli Broo...",Moussa Diabate,17,2,1,3,0,1,1,0,1,2,True,False,False,False,14.0,4433249,M. Diabate,https://a.espncdn.com/i/headshots/mens-college...,Forward,F,Wolverines,https://a.espncdn.com/i/teamlogos/ncaa/500/130...,130,MICH,00274c,2022,2,2022-02-05,1,4,0,0,0,0,0.0,HOME,2509,PUR,0.117647,10.00,0.358,76,10,Big_man,59.429,29.143,27.600,55.0,76,73.571,1,1,24.500,26.844,0,1,0
19216,Michigan,401364421,"{""DeVante' Jones"", 'Moussa Diabate', 'Eli Broo...",Caleb Houstan,31,0,2,2,3,0,0,0,1,11,True,False,False,False,22.0,4433623,C. Houstan,https://a.espncdn.com/i/headshots/mens-college...,Forward,F,Wolverines,https://a.espncdn.com/i/teamlogos/ncaa/500/130...,130,MICH,00274c,2022,2,2022-02-05,3,8,2,6,3,4,0.0,HOME,2509,PUR,0.354839,12.33,0.404,76,10,Big_man,59.429,29.143,27.600,55.0,76,73.571,1,1,24.500,26.844,0,1,0
19217,Michigan,401364421,"{""DeVante' Jones"", 'Moussa Diabate', 'Eli Broo...",Hunter Dickinson,36,0,2,2,2,0,2,4,4,28,True,False,False,False,1.0,4432180,H. Dickinson,https://a.espncdn.com/i/headshots/mens-college...,Center,C,Wolverines,https://a.espncdn.com/i/teamlogos/ncaa/500/130...,130,MICH,00274c,2022,2,2022-02-05,12,17,2,3,2,2,0.0,HOME,2509,PUR,0.777778,20.62,0.670,76,10,Big_man,59.429,29.143,27.600,55.0,76,73.571,1,1,24.500,26.844,0,1,0
19218,Michigan,401364421,"{""DeVante' Jones"", 'Moussa Diabate', 'Eli Broo...",DeVante' Jones,32,0,4,4,4,0,0,3,4,13,True,False,False,False,12.0,4397506,D. Jones,https://a.espncdn.com/i/headshots/mens-college...,Guard,G,Wolverines,https://a.espncdn.com/i/teamlogos/ncaa/500/130...,130,MICH,00274c,2022,2,2022-02-05,5,7,1,1,2,2,0.0,HOME,2509,PUR,0.406250,10.22,0.353,76,10,Guard,59.429,29.143,27.600,55.0,76,73.571,1,1,29.031,31.130,0,0,1
19219,Michigan,401364421,"{""DeVante' Jones"", 'Moussa Diabate', 'Eli Broo...",Eli Brooks,33,0,3,3,3,1,0,2,3,9,True,False,False,False,55.0,4277960,E. Brooks,https://a.espncdn.com/i/headshots/mens-college...,Guard,G,Wolverines,https://a.espncdn.com/i/teamlogos/ncaa/500/130...,130,MICH,00274c,2022,2,2022-02-05,3,6,1,3,2,2,0.0,HOME,2509,PUR,0.272727,9.89,0.284,76,10,Guard,59.429,29.143,27.600,55.0,76,73.571,1,1,29.031,31.130,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19296,Michigan,401409769,"{""DeVante' Jones"", 'Moussa Diabate', 'Eli Broo...",Jaron Faulds,2,0,0,0,0,0,0,0,0,0,False,False,False,False,44.0,4279471,J. Faulds,https://a.espncdn.com/i/headshots/mens-college...,Forward,F,Wolverines,https://a.espncdn.com/i/teamlogos/ncaa/500/130...,130,MICH,00274c,2022,2,2022-02-11,0,0,0,0,0,0,29.0,AWAY,2509,PUR,0.000000,1.00,0.325,82,27,Big_man,14.000,12.421,13.633,20.0,82,73.875,0,0,7.405,12.689,0,1,0
19297,Michigan,401409769,"{""DeVante' Jones"", 'Moussa Diabate', 'Eli Broo...",Ian Burns,2,0,0,0,0,0,0,0,0,0,False,False,False,False,13.0,4897424,I. Burns,https://a.espncdn.com/i/headshots/mens-college...,Guard,G,Wolverines,https://a.espncdn.com/i/teamlogos/ncaa/500/130...,130,MICH,00274c,2022,

Data Preprocessing

* Columns with "prev_" are the cumulative avgs shifted to obtain the averages before each game.
* Role = Starter/reserve
* Position = Converted all "Center" positions to "bigs" to have all 'forwards' and 'centers' the same, while having point guards and shooting guards as guards



In [38]:
#### Columns to use for model 
def transform_training_testing(dataframe):
    columns = ["prev_player_ppg","prev_player_points_per_min",	"game_number",
    "cum_average_opp_points_to_role","cum_average_opp_min_allowed_to_role",
    "prev_3_games_opp_min_allowed_to_role",
    "prev_3_games__opp_points_allowed_to_role",
    "prev_total_opp_ppg","Home","Starter_numeric",
    "prev_avg_pts_to_role_pos","prev_avg_min_to_role_position",
    "Big_man","Guard",
    "pts"]

    points_df_model =dataframe.loc[:,columns] 
    points_df_model = points_df_model.rename(columns = {"prev_player_ppg":"prev_avg_player_ppg","prev_player_points_per_min":"prev_avg_player_points_per_min"})
    points_df_model = points_df_model.dropna()
    return points_df_model

training_df_processed = transform_training_testing(training_df)
testing_df_processed = transform_training_testing(testing_df)

In [39]:
def convert_to_training_testing(training_df_processed,testing_df_processed):
    X_train= training_df_processed.loc[:, ~training_df_processed.columns.isin(['pts'])]               
    y_train = training_df_processed["pts"]
    X_test= testing_df_processed.loc[:, ~testing_df_processed.columns.isin(['pts'])]               
    y_test = testing_df_processed["pts"]

    # X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0)
    X_train_scaled = MinMaxScaler().fit_transform(X_train)
    X_test_scaled= MinMaxScaler().fit_transform(X_test)
    return X_train,y_train,X_test,y_test,X_train_scaled,X_test_scaled
X_train,y_train,X_test,y_test,X_train_scaled,X_test_scaled = convert_to_training_testing(training_df_processed,testing_df_processed)


#### Uncomment to test for overfitting
# kfold = KFold(n_splits=5)
# X = points_df_model.loc[:, ~points_df_model.columns.isin(['pts'])]               
# y = points_df_model["pts"]
# X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0)
# X_train_scaled = MinMaxScaler().fit_transform(X_train)
# X_test_scaled= MinMaxScaler().fit_transform(X_test)
# X_train

In [40]:
#### Linear Regression has the lowest MAE compared to the other models tested
linear_reg = LinearRegression().fit(X_train_scaled,y_train)
y_pred = linear_reg.predict(X_test_scaled)
mae = mean_absolute_error(y_test, y_pred)

model_name = "Linear reg"
print(model_name)
print("Mean Absolute Error: {}".format(mae))
pts_predictions_df = y_test.to_frame().rename(columns={"pts":"pts_label"})
pts_predictions_df['points_prediction'] = y_pred
merged_predictions = pd.merge(pts_predictions_df,sample_df_box_score_and_features,left_index= True, right_index= True)

merged_predictions.sample(20)
### Uncomment for sample team
# test_boxscore[test_boxscore.team_short_display_name=='Michigan']

Linear reg
Mean Absolute Error: 3.8515258948587414


,pts_label,points_prediction,team_short_display_name,game_id,starting_lineup,athlete_display_name,min,oreb,dreb,reb,ast,stl,blk,to,pf,pts,starter,ejected,did_not_play,active,athlete_jersey,athlete_id,athlete_short_name,athlete_headshot_href,athlete_position_name,athlete_position_abbreviation,team_name,team_logo,team_id,team_abbreviation,team_color,season,season_type,game_date,FGM,FGA,3PM,3PA,FTM,FTA,largest_lead,home_away,opponent_id,opponent_abbrev,points_per_min,prev_player_ppg,prev_player_points_per_min,game_total_points,game_number,Cleaned_position_bigs_guards,cum_average_opp_points_to_role,cum_average_opp_min_allowed_to_role,prev_3_games_opp_min_allowed_to_role,prev_3_games__opp_points_allowed_to_role,total_pts_allowed,prev_total_opp_ppg,Home,Starter_numeric,prev_avg_pts_to_role_pos,prev_avg_min_to_role_position,ATH,Big_man,Guard
20967,17,13.692785,Tennessee,401364934,"{'Josiah-Jordan James', 'Kennedy Chandler', 'B...",Kennedy Chandler,36,0,5,5,6,2,0,0,0,17,True,False,False,False,1.0,4432646,K. Chandler,https://a.espncdn.com/i/headshots/mens-college...,Guard,G,Volunteers,https://a.espncdn.com/i/teamlogos/ncaa/500/263...,2633,TENN,EE9627,2022,2,2022-02-16,5,9,3,5,4,4,20.0,AWAY,96,UK,0.472222,12.33,0.400,76,14,Guard,53.700,28.220,28.000,52.333,76,69.600,0,1,32.366,30.738,0,0,1
21528,2,11.283230,UT Martin,401373266,"{'Bernie Andre', 'KK Curry', 'KJ Simon', 'Chri...",Bernie Andre,18,0,3,3,0,0,0,1,0,2,True,False,False,False,21.0,4397303,B. Andre,https://a.espncdn.com/i/headshots/mens-college...,Forward,F,Skyhawks,https://a.espncdn.com/i/teamlogos/ncaa/500/263...,2630,UTM,FF6700,2022,2,2022-01-29,1,4,0,2,0,0,2.0,HOME,2057,BEL,0.111111,10.00,0.379,58,12,Big_man,53.667,27.200,27.200,53.667,58,71.667,1,1,19.667,23.812,0,1,0
19030,6,8.943599,Marshall,401372694,"{'David Early', 'Obinna Anochili-Killen', 'And...",David Early,30,0,5,5,2,2,0,0,3,6,True,False,False,False,23.0,4702695,D. Early,https://a.espncdn.com/i/headshots/mens-college...,Guard,G,Thundering Herd,https://a.espncdn.com/i/teamlogos/ncaa/500/276...,276,MRSH,00ae42,2022,2,2022-01-22,2,7,0,2,2,3,1.0,HOME,2226,FAU,0.200000,6.88,0.311,60,10,Guard,54.750,31.300,33.067,59.000,60,66.750,1,1,39.000,31.622,0,0,1
18834,1,1.889707,Kentucky,401364908,"{'TyTy Washington Jr.', 'Oscar Tshiebwe', 'Kei...",Lance Ware,6,1,0,1,0,0,2,1,2,1,False,False,False,False,55.0,4433152,L. Ware,https://a.espncdn.com/i/headshots/mens-college...,Forward,F,Wildcats,https://a.espncdn.com/i/teamlogos/ncaa/500/96.png,96,UK,005DAA,2022,2,2022-02-03,0,0,0,0,1,2,16.0,AWAY,238,VAN,0.166667,2.38,0.521,77,10,Big_man,12.167,13.486,14.028,15.000,77,65.833,0,0,6.808,10.384,0,1,0
18102,14,6.792761,Fairfield,401365338,"{'Chris Maidoh', 'Jake Wojcik', 'Taj Benning',...",Jesus Cruz,19,1,4,5,2,2,0,0,4,14,False,False,False,False,24.0,4279488,J. Cruz,https://a.espncdn.com/i/headshots/mens-college...,Guard,G,Stags,https://a.espncdn.com/i/teamlogos/ncaa/500/221...,2217,FAIR,000000,2022,2,2022-01-21,5,10,2,5,2,2,7.0,AWAY,2405,MONM,0.736842,7.62,0.351,58,17,Guard,17.200,12.850,10.300,14.667,58,65.800,0,0,13.182,14.950,0,0,1
17677,7,10.408324,Butler,401373575,"{'Bo Hodges', 'Bryce Nze', 'Chuck Harris', 'Aa...",Bryce Golden,30,1,2,3,4,1,0,0,1,7,True,False,False,False,33.0,4397026,B. Golden,https://a.espncdn.com/i/headshots/mens-college...,Forward,F,Bulldogs,https://a.espncdn.com/i/teamlogos/ncaa/500/208...,2086,BUT,0d1361,2022,2,2022-02-03,2,4,1,2,2,2,5.0,HOME,2752,XAV,0.233333,8.94,0.366,66,18,Big_man,53.500,29.275,31.667,57.667,66,68.250,1,1,21.848,27.376,0,1,0
19939,3,2.511614,Oklahoma State,401368373,"{'Moussa Cisse', 'Isaac Likekele', 'Bryce Will...",Tyreek Smith,22,2,6,8,2,1,1,1,1,3,False,False,False,False,23.0,4431764,T. Smith,https://a.espncdn.com/i/headshots/mens-college...,Forward,F,Cowboys,https://a.espncdn.com/i/teamlogos/ncaa/500/197...,197,OKST,FF6500,2022,2,2022-01-15,1,3,0,0,1,2,18.0,HOME,239,BAY,0.136364,3.60,0.362,61,6,Big_man,10.500,11.400,12.133,12.000,61,59.250,1,0,5.122,10.051,0,1,0
17494,10,10.020184,Belmo

Prediction Visualization

In [41]:
def display_prediction_graph(player_pred):#line= True, bars=True,head=True,logo= True):
    # player_pred = merged_predictions[merged_predictions.athlete_display_name == player]

    line = alt.Chart(player_pred).mark_line(strokeDash=[1,1],color= 'black').encode(
      x='game_date:T',
      y=alt.Y('points_prediction'),
      tooltip = ["game_date:T",'points_prediction','pts',"min","starter"]
      )
    bars = alt.Chart(player_pred).mark_bar().encode(
      x=alt.X('game_date:T', title = 'Game Date'),
      y=alt.Y('pts', title = "Points"),
      color = alt.Color('team_color', legend = None),
      tooltip = ["game_date:T",'points_prediction','pts',"min","starter"]
      )
    try:
        logo = alt.Chart(player_pred).mark_image().encode(
          x=alt.value(500), x2=alt.value(600),  # pixels from left
          y=alt.value(0), y2=alt.value(75),  # pixels from top
          url='team_logo'
        )
    except:
        return (line+bars)

    try:
        head_shot_image = alt.Chart(player_pred).mark_image().encode(
        x=alt.value(500), x2=alt.value(600),  # from left
        y=alt.value(400), y2=alt.value(500),  # from top
        url='athlete_headshot_href'
        ).properties(
            width = 500,
            height  =500,
            title = "Points (Bars) and Predicted Points (Dashed line)"
        )
        return (line+bars+logo+head_shot_image)
    except:
        return (line+bars+logo)  

In [44]:
# team = 'Michigan'

### Uncomment other players
player = 'Hunter Dickinson'
# player = 'Paolo Banchero'
# player = "Ochai Agbaji"
### Filter on team, player, 
# team_predictions = merged_predictions[merged_predictions.team_short_display_name==team] 

#### If statements when the images are null. 
player_pred = merged_predictions[merged_predictions.athlete_display_name == player]
display_prediction_graph(player_pred)

alt.LayerChart(...)

Other models considered

In [ ]:
# #### Uncomment below for overfitting
# # name_model = "Lasso"
# # scorer_name = "mean abs error"
# # scorer_func= mean_absolute_error
# # print(name_model)
# # alphas= np.arange(0.01,.1,.02)
# # param_grid = dict(alpha =alphas)
# # scorer = make_scorer(scorer_func)
# # grid = GridSearchCV(Lasso(),param_grid=param_grid, scoring = scorer,cv = 5)
# # grid_result = grid.fit(X_train_scaled, y_train)
# # # print("Best Score: %f \nparameters %s" % (grid_result.best_score_, grid_result.best_params_))
# # print("best param: {}".format(grid_result.best_params_))

# # best_alpha = list(grid_result.best_params_.values())[0]
# # result = cross_val_score(Lasso(alpha=best_alpha),X_train_scaled,y_train, cv = kfold, scoring=scorer)
# # print("Mean {} score using cross validation: {}".format(str(scorer_name),result.mean()))
# # # grid_result.get_params
# Lasso_model = (Lasso(alpha =  0.0899).fit(X_train_scaled,y_train))
# y_pred = Lasso_model.predict(X_test_scaled)
# mae = mean_absolute_error(y_test, y_pred)

# model_name = "Lasso"
# print(model_name)
# print("Mean Absolute Error: {}".format(mae))



# # merged_predictions[merged_predictions.team_short_display_name=='Michigan']

# ### uncomment for k-fold and train_test_split
# # pts_predictions_df = y_test.to_frame().rename(columns={"pts":"pts_label"})
# # pts_predictions_df['points_prediction'] = y_pred
# # sampled_player_games = pts_predictions_df.sample(5000,random_state = 150)
# # merged_predictions = pd.merge(sampled_player_games,box_scores_and_features,left_index= True, right_index= True, how = "left")
# # merged_predictions[merged_predictions.team_short_display_name=='Michigan']

In [ ]:
# # name_model = "Ridge"
# # print(name_model)
# # scorer_name = "mean abs error"
# # scorer_func= mean_absolute_error
# # alphas= np.arange(0.1,100,10)
# # param_grid = dict(alpha =alphas)
# # scorer = make_scorer(scorer_func)
# # grid = GridSearchCV(Ridge(),param_grid=param_grid, scoring = scorer,cv = 5)
# # grid_result = grid.fit(X_train_scaled, y_train)
# # # print("Best Score: %f \nparameters %s" % (grid_result.best_score_, grid_result.best_params_))
# # print("best param: {}".format(grid_result.best_params_))

# # best_alpha = list(grid_result.best_params_.values())[0]
# # result = cross_val_score(Ridge(alpha=best_alpha),X_train_scaled,y_train, cv = kfold, scoring=scorer)
# # print("Mean {} score using cross validation: {}".format(scorer_name,result.mean()))

# Ridge_model = (Ridge(alpha =  90.1).fit(X_train_scaled,y_train))
# y_pred = Ridge_model.predict(X_test_scaled)
# mae = mean_absolute_error(y_test, y_pred)

# model_name = "Ridge"
# print(model_name)
# print("Mean Absolute Error: {}".format(mae))

In [ ]:
# #### Use less data to gridsearch params when using train_test_split on all seasons

# columns = ["prev_player_ppg","prev_player_points_per_min",	"game_number",
# "cum_average_opp_points_to_role","cum_average_opp_min_allowed_to_role",
# "prev_3_games_opp_min_allowed_to_role",
# "prev_3_games__opp_points_allowed_to_role",
# "prev_total_opp_ppg","Home","Starter_numeric",	
# "prev_avg_pts_to_role_pos","prev_avg_min_to_role_position",
# "Big_man","Guard",
# "pts"]
# sample_df = box_scores_and_features[box_scores_and_features["season"]==2019][:5000]
# sample_points_df_model =sample_df.loc[:,columns] 
# #### Previous columns are the cumulative avgs shifted to obtain the averages before each game. ### Column names might need to be renamed 
# #### Role = Starter/reserve
# ### Position - Converted all "Center" positions to "bigs" to have all 'forwards' and 'centers' the same, while having point guards and shooting guards as guards
# sample_points_df_model = sample_points_df_model.rename(columns = {"prev_player_ppg":"prev_avg_player_ppg","prev_player_points_per_min":"prev_avg_player_points_per_min"})

# sample_points_df_model = sample_points_df_model.dropna()
# kfold = KFold(n_splits=5)
# sample_X = sample_points_df_model.loc[:, ~sample_points_df_model.columns.isin(['pts'])]               
# sample_y = sample_points_df_model["pts"]
# sample_X_train, sample_X_test, sample_y_train, sample_y_test = train_test_split(sample_X,sample_y,random_state=0)
# sample_X_train_scaled = MinMaxScaler().fit_transform(sample_X_train)
# sample_X_test_scaled= MinMaxScaler().fit_transform(sample_X_test)
# sample_X_train

In [ ]:
# from sklearn.ensemble import GradientBoostingRegressor

# # name_model = "GradientBoostingRegressor"
# # scorer_name = "mean abs error"
# # scorer_func= mean_absolute_error
# # scorer = make_scorer(scorer_func)
# param_grid = {
#     "loss":["absolute_error"],
#     "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
#     # "min_samples_split": np.arange(1,7,2),
#     # "min_samples_leaf":  np.arange(1,7,2),
#     "max_depth":[8],
#     # "max_features":["log2","sqrt"],
#     "subsample":[.25,0.5,.75, 1.0],
#     }
# grid = GridSearchCV(GradientBoostingRegressor(random_state=0),param_grid=param_grid, scoring = scorer,cv = kfold)
# grid_result = grid.fit(sample_X_train_scaled, sample_y_train)
# # print("Best Score: %f \nparameters %s" % (grid_result.best_score_, grid_result.best_params_))
# print("best param: {}".format(grid_result.best_params_))
# # GradientBoostingModel = GradientBoostingRegressor()
# # GradientBoostingModel = GradientBoostingModel.set_params(**grid.best_params_).fit(X_train_scaled,y_train)
# # best_depth = list(grid_result.best_params_.values())[0]
# # result = cross_val_score(RandomForestRegressor(max_depth=best_depth,random_state=0),X_train_scaled,y_train, cv = kfold, scoring=scorer)
# # print("Mean {} score using cross validation: {}".format(scorer_name,result.mean()))

# GradientBoostingModel = GradientBoostingRegressor()
# GradientBoostingModel = GradientBoostingModel.set_params(**grid.best_params_).fit(X_train_scaled,y_train)
# y_pred = GradientBoostingModel.predict(X_test_scaled)
# mae = mean_absolute_error(y_test, y_pred)
# # mse = mean_squared_error(y_test, y_pred)
# # print(depth, r2_train, r2_test,mae)
# model_name = "GradientBoostingModel"
# print(model_name)
# print("Mean Absolute Error: {}".format(mae))

In [ ]:
# # name_model = "RandomForestRegressor"
# # scorer_name = "mean abs error"
# # depths = np.arange(1,10,1)
# # param_grid = dict(max_depth =depths)
# # scorer = make_scorer(r2_score)
# # grid = GridSearchCV(RandomForestRegressor(random_state=0),param_grid=param_grid, scoring = scorer,cv = kfold)
# # grid_result = grid.fit(X_train_scaled, y_train)
# # # print("Best Score: %f \nparameters %s" % (grid_result.best_score_, grid_result.best_params_))
# # print("best param: {}".format(grid_result.best_params_))
# # best_depth = list(grid_result.best_params_.values())[0]
# # result = cross_val_score(RandomForestRegressor(max_depth=best_depth,random_state=0),X_train_scaled,y_train, cv = kfold, scoring=scorer)
# # print("Mean {} score using cross validation: {}".format(scorer_name,result.mean()))

# # for num in np.arange(8,12,1):
# # print(num)
# RFmodel = (RandomForestRegressor(max_depth=9,random_state=0).fit(X_train_scaled,y_train))
# y_pred = RFmodel.predict(X_test_scaled)
# mae = mean_absolute_error(y_test, y_pred)
# # mse = mean_squared_error(y_test, y_pred)
# # print(depth, r2_train, r2_test,mae)
# model_name = "Random Forest"
# print(model_name)
# print("Mean Absolute Error: {}".format(mae))

# pts_predictions_df = y_test.to_frame().rename(columns={"pts":"pts_label"})
# pts_predictions_df['points_prediction'] = y_pred
# merged_predictions = pd.merge(pts_predictions_df,box_scores_and_features,left_index= True, right_index= True,how = 'left')
# merged_predictions[merged_predictions.team_short_display_name=='Michigan']
